Design and implement a Complete University Management System in Python using SQLite and functions (def) with the following requirements:

1. Database (SQLite) Create the following tables: Departments (dept_id, dept_name, hod_name) Students (student_id, name, class, dept_id) Subjects (subject_id, subject_name, dept_id) Marks (mark_id, student_id, subject_id, marks) Insert: At least 2 departments, 5 students, 4 subjects, and marks for each student in all subjects.
2. Python Functions Implement the following: def add_student(name, class, dept_id) def add_subject(subject_name, dept_id) def add_marks(student_id, subject_id, marks) def show_students() → Display students with department name (JOIN) def show_subjects() → Display all subjects with their department def show_marks() → Display each student’s marks with subject names (JOIN) def student_report(student_id) → Show subject-wise marks, total, average, grade, pass/fail def toppers() → Show topper in each subject + overall topper def department_summary() → Show average marks department-wise (JOIN + GROUP BY) def unique_subjects() → Return unique subjects as a set


In [1]:
import sqlite3

In [2]:
def init_db():
    con=sqlite3.connect("Uni.db")
    cursor=con.cursor()

    cursor.execute("""
    CREATE TABLE Departments(
    dept_id INTEGER PRIMARY KEY AUTOINCREMENT,
    dept_name TEXT,
    hod_name TEXT)
    """)

    cursor.execute("""
    CREATE TABLE Students(
    student_id INTEGER PRIMARY KEY,
    name TEXT,
    class INTEGER,
    dept_id INTEGER,
    FOREIGN KEY (dept_id) REFERENCES Departments(dept_id))
    """)

    cursor.execute("""
    CREATE TABLE Subjects(
    subject_id INTEGER PRIMARY KEY,
    subject_name TEXT,
    dept_id INTEGER,
    FOREIGN KEY (dept_id) REFERENCES Departments(dept_id))
    """)

    cursor.execute("""
    CREATE TABLE Marks(
    mark_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_id INTEGER,
    subject_id INTEGER,
    marks INTEGER,
    FOREIGN KEY (student_id) REFERENCES Students(student_id),
    FOREIGN KEY (subject_id) REFERENCES Subjects(subject_id))
    """)
    
    Departments=[('Business', 'Luke'),
             ('Science', 'Alex')]
    cursor.executemany("INSERT INTO Departments (dept_name, hod_name) VALUES (?,?)", Departments)
    
    Students=[(1001, 'Joe', 2, 1),
          (1002, 'Lily', 1, 1),
          (1003, 'Manny', 3, 1),
          (1004, 'Dylan', 2, 2,),
          (1005, 'Ruben', 3, 2)]
    cursor.executemany("INSERT INTO Students (student_id, name, class, dept_id) VALUES (?,?,?,?)", Students)
    
    Subjects=[(101, 'Math', 1),
          (102, 'Physics', 2),
          (103, 'Chemistry', 2),
          (104, 'English', 1)]
    cursor.executemany("INSERT INTO Subjects (subject_id, subject_name, dept_id) VALUES (?,?,?)", Subjects)
    
    Marks=[(1001, 101, 83), (1001, 102, 87), (1001, 103, 95), (1001, 104, 86),
       (1002, 101, 90), (1002, 102, 84), (1002, 103, 89), (1002, 104, 92),
       (1003, 101, 74), (1003, 102, 80), (1003, 103, 79), (1003, 104, 85),
       (1004, 101, 81), (1004, 102, 77), (1004, 103, 88), (1004, 104, 85),
       (1005, 101, 76), (1005, 102, 86), (1005, 103, 83), (1005, 104, 73)]
    cursor.executemany("INSERT INTO Marks (student_id, subject_id, marks) VALUES (?,?,?)", Marks)
    
    con.commit()
    return con

In [5]:
import sqlite3
db_name = "uni.db"
def get_connection():
    return sqlite3.connect(db_name)
    
def add_student(name, class_, dept_id):
    con = get_connection()
    cursor = con.cursor()
    cursor.execute("INSERT INTO Students (name, class, dept_id) VALUES (?, ?, ?)", (name, class_, dept_id))
    con.commit()

def add_subject(subject_name, dept_id):
    con = get_connection()
    cursor = con.cursor()
    cursor.execute("INSERT INTO Subjects (subject_name, dept_id) VALUES (?, ?)", (subject_name, dept_id))
    con.commit()

def add_marks(student_id, subject_id, marks):
    con = get_connection()
    cursor = con.cursor()
    cursor.execute("INSERT INTO Marks (student_id, subject_id, marks) VALUES (?, ?, ?)", (student_id, subject_id, marks))
    con.commit()

def show_students():
    con = get_connection()
    cursor = con.cursor()
    cursor.execute("""
        SELECT s.student_id, s.name, s.class, d.dept_name
        FROM Students s
        JOIN Departments d ON s.dept_id = d.dept_id
    """)
    rows = cursor.fetchall()
    print("Students: ")
    for r in rows:
        print(r)

def show_subjects():
    con = get_connection()
    cursor = con.cursor()
    cursor.execute("""
        SELECT sub.subject_id, sub.subject_name, d.dept_name
        FROM Subjects sub
        JOIN Departments d ON sub.dept_id = d.dept_id
    """)
    rows = cursor.fetchall()
    print("Subjects: ")
    for r in rows:
        print(r)

def show_marks():
    con = get_connection()
    cursor = con.cursor()
    cursor.execute("""
        SELECT s.name, sub.subject_name, m.marks
        FROM Marks m
        JOIN Students s ON m.student_id = s.student_id
        JOIN Subjects sub ON m.subject_id = sub.subject_id
    """)
    rows = cursor.fetchall()
    print("Marks:")
    for r in rows:
        print(r)


def student_report(student_id):
    con = get_connection()
    cursor = con.cursor()
    cursor.execute("""
        SELECT sub.subject_name, m.marks
        FROM Marks m
        JOIN Subjects sub ON m.subject_id = sub.subject_id
        WHERE m.student_id = ?
    """, (student_id,))
    results = cursor.fetchall()

    if not results:
        print("No records found.")
        return

    total = sum([r[1] for r in results])
    avg = total / len(results)

    # Grade system
    if avg >= 90:
        grade = "A+"
    elif avg >= 75:
        grade = "A"
    elif avg >= 60:
        grade = "B"
    elif avg >= 50:
        grade = "C"
    else:
        grade = "F"

    status = "Pass" if avg >= 50 else "Fail"

    print(f"\nReport for Student ID {student_id}:")
    for sub, mark in results:
        print(f"{sub}: {mark}")
    print(f"Total: {total}, Average: {avg:.2f}, Grade: {grade}, Status: {status}")

def toppers():
    con = get_connection()
    cursor = con.cursor()

    cursor.execute("""
        SELECT sub.subject_name, s.name, MAX(m.marks)
        FROM Marks m
        JOIN Students s ON m.student_id = s.student_id
        JOIN Subjects sub ON m.subject_id = sub.subject_id
        GROUP BY sub.subject_id
    """)
    subject_toppers = cursor.fetchall()

    print("\nSubject-wise Toppers:")
    for row in subject_toppers:
        print(f"{row[0]} -> {row[1]} ({row[2]})")

    cursor.execute("""
        SELECT s.name, SUM(m.marks) as total
        FROM Marks m
        JOIN Students s ON m.student_id = s.student_id
        GROUP BY s.student_id
        ORDER BY total DESC
        LIMIT 1
    """)
    overall_topper = cursor.fetchone()
    print(f"\nOverall Topper: {overall_topper[0]} ({overall_topper[1]} marks)")

def department_summary():
    con = get_connection()
    cursor = con.cursor()
    cursor.execute("""
        SELECT d.dept_name, AVG(m.marks) as avg_marks
        FROM Marks m
        JOIN Students s ON m.student_id = s.student_id
        JOIN Departments d ON s.dept_id = d.dept_id
        GROUP BY d.dept_id
    """)
    rows = cursor.fetchall()
    print("\nDepartment Summary (Average Marks):")
    for r in rows:
        print(f"{r[0]}: {r[1]:.2f}")

def unique_subjects():
    con = get_connection()
    cursor = con.cursor()
    cursor.execute("SELECT DISTINCT subject_name FROM Subjects")
    rows = cursor.fetchall()
    con.close()
    
    return {r[0] for r in rows}


In [6]:
show_students()

Students: 
(1001, 'Joe', 2, 'Business')
(1002, 'Lily', 1, 'Business')
(1003, 'Manny', 3, 'Business')
(1004, 'Dylan', 2, 'Science')
(1005, 'Ruben', 3, 'Science')


In [7]:
show_subjects()

Subjects: 
(101, 'Math', 'Business')
(102, 'Physics', 'Science')
(103, 'Chemistry', 'Science')
(104, 'English', 'Business')


In [8]:
show_marks()

Marks:
('Joe', 'Math', 83)
('Joe', 'Physics', 87)
('Joe', 'Chemistry', 95)
('Joe', 'English', 86)
('Lily', 'Math', 90)
('Lily', 'Physics', 84)
('Lily', 'Chemistry', 89)
('Lily', 'English', 92)
('Manny', 'Math', 74)
('Manny', 'Physics', 80)
('Manny', 'Chemistry', 79)
('Manny', 'English', 85)
('Dylan', 'Math', 81)
('Dylan', 'Physics', 77)
('Dylan', 'Chemistry', 88)
('Dylan', 'English', 85)
('Ruben', 'Math', 76)
('Ruben', 'Physics', 86)
('Ruben', 'Chemistry', 83)
('Ruben', 'English', 73)


In [9]:
student_report(1002)


Report for Student ID 1002:
Math: 90
Physics: 84
Chemistry: 89
English: 92
Total: 355, Average: 88.75, Grade: A, Status: Pass


In [10]:
toppers()


Subject-wise Toppers:
Math -> Lily (90)
Physics -> Joe (87)
Chemistry -> Joe (95)
English -> Lily (92)

Overall Topper: Lily (355 marks)


In [11]:
department_summary()


Department Summary (Average Marks):
Business: 85.33
Science: 81.12


In [12]:
print(unique_subjects())

{'English', 'Math', 'Chemistry', 'Physics'}
